<H1>Step 1: Recueillir les données (Data Collection)</H1>

In [7]:
import pandas as pd
#charger le fichier Excel 
file_path= '/home/lamsiyehkhawla/Downloads/sales.xlsx'
sales_df=pd.read_excel(file_path)
print("Colonnes du fichier :", sales_df.columns.tolist())



Colonnes du fichier : ['OrderNumber', 'OrderDate', 'Ship Date', 'Customer Name Index', 'Channel', 'Currency Code', 'Warehouse Code', 'Delivery Region Index', 'Product Description Index', 'Order Quantity', 'Unit Selling Price', 'Unit Cost']


In [10]:
import pandas as pd
#charger le fichier Excel 
file_path= '/home/lamsiyehkhawla/Downloads/sales.xlsx'
sales_df=pd.read_excel(file_path)

# Netooyage des colonnes
sales_df.columns = sales_df.columns.str.strip()
#renommer les colonnes à utiliser 
sales_df= sales_df.rename(columns={
    'OrderDate': 'Date',
    'Order Quantity': 'Order_Quantity',
    'Unit Selling Price': 'Unit_Selling_Price',
    'Unit Cost': 'Unit_cost',
    'Customer Name Index': 'Customer',
    'Product Description Index': 'Product',
    'Delivery Region Index': 'Region'
})
# Calculer les champs nécessaires 
sales_df['Sales']=sales_df['Order_Quantity'] * sales_df['Unit_Selling_Price']
sales_df['Cost'] = sales_df['Order_Quantity'] * sales_df['Unit_cost']
sales_df['Profit'] = sales_df['Sales'] - sales_df['Cost']
print(sales_df.head())

    OrderNumber       Date  Ship Date  Customer      Channel Currency Code  \
0   SO - 000225 2017-01-01 2017-01-13        28    Wholesale           NZD   
1  SO - 0003378 2017-01-01 2017-01-06         7  Distributor           NZD   
2  SO - 0003901 2017-01-01 2017-01-05        12    Wholesale           NZD   
3  SO - 0005126 2017-01-01 2017-01-17         5    Wholesale           USD   
4  SO - 0005614 2017-01-01 2017-01-07        27       Export           NZD   

  Warehouse Code  Region  Product  Order_Quantity  Unit_Selling_Price  \
0         AXW291      71       11               6              2499.1   
1         AXW291      54        7              11              2351.7   
2         AXW291      58       13               5              1728.6   
3         AXW291      29        7               6               978.2   
4         AXW291      31        6               7              2338.3   

   Unit_cost    Sales       Cost     Profit  
0   1824.343  14994.6  10946.058   4048.542  


<H1> Step 2: Nettoyage et transformation (ETL)</H1> 


In [11]:
#Supprimer les doublons
sales_df=sales_df.drop_duplicates()

print("Valeurs manquantes par colonne :")
print(sales_df.isna().sum())
sales_df = sales_df.dropna(subset=['Date','Sales','Profit','Cost','Order_Quantity'])

sales_df['Date'] = pd.to_datetime(sales_df['Date'])

print(sales_df.dtypes)

Valeurs manquantes par colonne :
OrderNumber           0
Date                  0
Ship Date             0
Customer              0
Channel               0
Currency Code         0
Warehouse Code        0
Region                0
Product               0
Order_Quantity        0
Unit_Selling_Price    0
Unit_cost             0
Sales                 0
Cost                  0
Profit                0
dtype: int64
OrderNumber                   object
Date                  datetime64[ns]
Ship Date             datetime64[ns]
Customer                       int64
Channel                       object
Currency Code                 object
Warehouse Code                object
Region                         int64
Product                        int64
Order_Quantity                 int64
Unit_Selling_Price           float64
Unit_cost                    float64
Sales                        float64
Cost                         float64
Profit                       float64
dtype: object


<H1>Step 3: Créer un DataFrame des dates</H1>

In [22]:
#Extraire les dates min et max 
min_date = sales_df['Date'].min()
max_date = sales_df['Date'].max()

#Créer la table des dates 
#Genrer les ates entre la date min et max d'une maniere quotidienne 
date_range = pd.date_range(start=min_date, end=max_date,freq='D')
#DataFrame contiendra une seule colonne 'Date' qui contient toutes les dates générées dans date_range.
date_df = pd.DataFrame({'Date': date_range})
#Cela extrait l'année de chaque date dans la colonne 'Date' et la place dans une nouvelle colonne 'Year'.
date_df['Year'] = date_df['Date'].dt.year
#ela extrait le mois (sous forme de nombre de 1 à 12) et l'ajoute dans la colonne 'Month'
date_df['Month'] = date_df['Date'].dt.month
#Cela extrait le nom complet du mois (par exemple, "janvier", "février") et l'ajoute dans la colonne 
date_df['Month_Name'] = date_df['Date'].dt.strftime('%B')
#Cela détermine le trimestre de l'année (1, 2, 3 ou 4) et l'ajoute dans la colonne 'Quarter'.
date_df['Quarter'] = date_df['Date'].dt.quarter
#ela extrait le nom du jour de la semaine (par exemple, "lundi", "mardi") et l'ajoute dans la colonne 
date_df['Weekday'] = date_df['Date'].dt.day_name()

print(date_df.head())


        Date  Year  Month Month_Name  Quarter    Weekday
0 2017-01-01  2017      1    January        1     Sunday
1 2017-01-02  2017      1    January        1     Monday
2 2017-01-03  2017      1    January        1    Tuesday
3 2017-01-04  2017      1    January        1  Wednesday
4 2017-01-05  2017      1    January        1   Thursday


<H1>Step 4: Créer les mesures et indicateurs</H1>

In [32]:
sales_df_copy = sales_df.copy()

#Ajouter colonne annee et mois
sales_df_copy['Year'] = sales_df_copy['Date'].dt.year
sales_df_copy['Month'] = sales_df_copy['Date'].dt.month
sales_df_copy['Month_Name'] = sales_df_copy['Date'].dt.strftime('%B')


print(sales_df_copy[['Date','Year','Month','Month_Name']].head())

        Date  Year  Month Month_Name
0 2017-01-01  2017      1    January
1 2017-01-01  2017      1    January
2 2017-01-01  2017      1    January
3 2017-01-01  2017      1    January
4 2017-01-01  2017      1    January


<H2>Total Sales</H2>



In [42]:
total_sales = sales_df_copy['Sales'].sum()
print(f"Total sales : {total_sales:.2f} MAD")

Total sales : 154573140.60 MAD


<h2>Total Sales PY (Previous Year)</h2>

In [44]:
# contiendra un nouveau DataFrame avec deux colonnes Year et Sales
#Le résultat est un DataFrame propre où chaque ligne contient une année et le total des ventes pour cette année.
sales_by_year = sales_df_copy.groupby('Year')['Sales'].sum().reset_index()

print("Ventes par année")
print(sales_by_year)

Ventes par année
   Year       Sales
0  2017  52580534.7
1  2018  53463661.7
2  2019  48528944.2
